NETTOYAGE / FILTRAGE

In [1]:
import pandas as pd

In [ ]:
# Name Basics

'''
Colonnes à supprimer : Aucune
Lignes à supprimer:
    - knownfortitle = /N
    - primaryprofession = /N
'''

In [ ]:
# Title Akas

'''
Colonnes à supprimer : 
Lignes à supprimer: 
    - Faire un groupby par ID de film
    - A voir plus tard pour filtrer avec un contains sur language et/ou région
'''

In [ ]:
# Title Basics

'''
Colonnes à supprimer : 
Lignes à supprimer: 
    - Titletype = movie
    - A voir pour filtrer par années par la suite
    - A voir pour filtrer selon durée des films (trop courts ou trop longs)
    - isAdult = 1 et genre = adult (uniquement)
    - Vérifier classification de Basic instinct

'''

In [ ]:
# Title Crew

'''
RAS
'''

In [ ]:
# Title Episodes

'''
A voir : concatener numberEpisode et numberSeason
'''

In [ ]:
# Title Principals

'''
Colonnes à supprimer : job, character
'''

In [ ]:
# Title Ratings

'''
RAS
'''

In [4]:
# TMDB Full

link = "./BD/tmdb_full.csv"
df_tmdb_full = pd.read_csv(link)

df_tmdb_full['fr?'] = df_tmdb_full['spoken_languages'].apply(lambda x: 'fr' in x)
df_fr = df_tmdb_full[df_tmdb_full['fr?']]

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_14076/1416205938.py:4: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmdb_full = pd.read_csv(link)


In [9]:
link = "./BD/title.akas.tsv.gz"
df_title_akas = pd.read_csv(link, compression = 'gzip', sep = '\t')

In [21]:
df_title_akas = df_title_akas[(df_title_akas['region'] == 'FR') | (df_title_akas['language'] == 'fr')]

In [22]:
df_title_akas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4939892 entries, 12 to 50417999
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  int64 
dtypes: int64(2), object(6)
memory usage: 339.2+ MB


In [11]:
# JOIN Title.Akas

df_TMDB_AKAS = pd.merge(df_fr,
                        df_title_akas[['titleId', 'region', 'language']],
                        how = 'inner',
                        right_on = df_title_akas['titleId'],
                        left_on = df_fr['imdb_id'])

df_TMDB_AKAS

,key_0,adult,backdrop_path,budget,genres,homepage,id,imdb_id,original_language,original_title,...,title,video,vote_average,vote_count,production_companies_name,production_companies_country,fr?,titleId,region,language
0,tt0055747,False,/uJlc4aNPF3Y8yAqahJTKBwgwPVW.jpg,0,"['Drama', 'Romance']",NaN,8773,tt0055747,fr,L'Amour à vingt ans,...,Love at Twenty,False,6.7,41,"['Ulysse Productions', 'Unitec Films', 'Cinese...","['', 'NZ', 'IT', 'JP', 'DE', 'PL', '']",True,tt0055747,\N,\N
1,tt0055747,False,/uJlc4aNPF3Y8yAqahJTKBwgwPVW.jpg,0,"['Drama', 'Romance']",NaN,8773,tt0055747,fr,L'Amour à vingt ans,...,Love at Twenty,False,6.7,41,"['Ulysse Productions', 'Unitec Films', 'Cinese...","['', 'NZ', 'IT', 'JP', 'DE', 'PL', '']",True,tt0055747,DK,\N
2,tt0055747,False,/uJlc4aNPF3Y8yAqahJTKBwgwPVW.jpg,0,"['Drama', 'Romance']",NaN,8773,tt0055747,fr,L'Amour à vingt ans,...,Love at Twenty,False,6.7,41,"['Ulysse Productions', 'Unitec Films', 'Cinese...","['', 'NZ', 'IT', 'JP', 'DE', 'PL', '']",True,tt0055747,RO,\N
3,tt0055747,False,/uJlc4aNPF3Y8yAqahJTKBwgwPVW.jpg,0,"['Drama', 'Romance']",NaN,8773,tt0055747,fr,L'Amour à vingt ans,...,Love at Twenty,False,6.7,41,"['Ulysse Productions', 'Unitec Films', 'Cinese...","['', 'NZ', 'IT', 'JP', 'DE', 'PL', '']",True,tt0055747,ES,\N
4,tt0055747,False,/uJlc4aNPF3Y8yAqahJTKBwgwPVW.jpg,0,"['Drama', 'Romance']",NaN,8773,tt0055747,fr,L'Amour à vingt ans,...,Love at Twenty,False,6.7,41,"['Ulysse Productions', 'Unitec Films', 'Cinese...","['', 'NZ', 'IT', 'JP', 'DE', 'PL', '']",True,tt0055747,XWW,en


In [18]:
# df_TMDB_AKAS['fr?'] = df_TMDB_AKAS['spoken_languages'].apply(lambda x: 'fr' in x)
df_fr2 = df_TMDB_AKAS[df_TMDB_AKAS['region'] == 'FR']
df_fr2.head()


,key_0,adult,backdrop_path,budget,genres,homepage,id,imdb_id,original_language,original_title,...,title,video,vote_average,vote_count,production_companies_name,production_companies_country,fr?,titleId,region,language
13,tt0055747,False,/uJlc4aNPF3Y8yAqahJTKBwgwPVW.jpg,0,"['Drama', 'Romance']",NaN,8773,tt0055747,fr,L'Amour à vingt ans,...,Love at Twenty,False,6.700,41,"['Ulysse Productions', 'Unitec Films', 'Cinese...","['', 'NZ', 'IT', 'JP', 'DE', 'PL', '']",True,tt0055747,FR,\N
14,tt0055747,False,/uJlc4aNPF3Y8yAqahJTKBwgwPVW.jpg,0,"['Drama', 'Romance']",NaN,8773,tt0055747,fr,L'Amour à vingt ans,...,Love at Twenty,False,6.700,41,"['Ulysse Productions', 'Unitec Films', 'Cinese...","['', 'NZ', 'IT', 'JP', 'DE', 'PL', '']",True,tt0055747,FR,\N
83,tt0266697,False,/lVy5Zqcty2NfemqKYbVJfdg44rK.jpg,30000000,"['Action', 'Crime']",http://www.miramax.com/movie/kill-bill-volume-1,24,tt0266697,en,Kill Bill: Vol. 1,...,Kill Bill: Vol. 1,False,7.970,15748,"['Miramax', 'A Band Apart', 'Super Cool ManChu']","['US', 'US', '']",True,tt0266697,FR,\N
185,tt0078788,False,/9Qs9oyn4iE8QtQjGZ0Hp2WyYNXT.jpg,31500000,"['Drama', 'War']",http://www.apocalypsenow.com,28,tt0078788,en,Apocalypse Now,...,Apocalypse Now,False,8.281,7199,"['United Artists', 'American Zoetrope']","['US', 'US']",True,tt0078788,FR,\N
257,tt0114746,False,/1IWaKG7AWiYMhADxhGtnElDJAGI.jpg,29000000,"['Science Fiction', 'Thriller', 'Mystery']",NaN,63,tt0114746,en,Twelve Monkeys,...,Twelve Monkeys,False,7.596,7444,"['Universal Pictures', 'Atlas Entertainment', ...","['US', 'US', '']",True,tt0114746,FR,\N


In [8]:
df_TMDB_AKAS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240225 entries, 0 to 240224
Data columns (total 30 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   key_0                         240225 non-null  object 
 1   adult                         240225 non-null  bool   
 2   backdrop_path                 196398 non-null  object 
 3   budget                        240225 non-null  int64  
 4   genres                        240225 non-null  object 
 5   homepage                      38312 non-null   object 
 6   id                            240225 non-null  int64  
 7   imdb_id                       240225 non-null  object 
 8   original_language             240225 non-null  object 
 9   original_title                240225 non-null  object 
 10  overview                      232270 non-null  object 
 11  popularity                    240225 non-null  float64
 12  poster_path                   233915 non-nul